In [1]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

C:\Users\TUONGNGUSO03\AppData\Roaming\Python\Python38\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\TUONGNGUSO03\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [49]:
candidate_labels = ["animals", "animals activity", "water", "scenery", "weather", "human activity", "language", "plants"]

In [14]:
import pandas as pd
words = pd.read_csv("words.csv")
docs = list(words["Definitions"])

In [52]:
results = pd.DataFrame([], columns = candidate_labels)
for doc in docs:
    print(doc)
    scores = classifier(doc, candidate_labels, )["scores"]
    labels = classifier(doc, candidate_labels, )["labels"]
    results = pd.concat([results, pd.DataFrame(scores).transpose().set_axis(labels=labels, axis= 1)], axis = 0)

final particle
trousers, skirt
clothes
person who laughs easily
edible snail
delirious 


KeyboardInterrupt: 

In [53]:
results

,animals,animals activity,water,scenery,weather,human activity,language,plants
0,0.038399,0.073731,0.098784,0.039909,0.046750,0.086811,0.586388,0.029228
0,0.016083,0.047442,0.028530,0.030171,0.027242,0.634231,0.201780,0.014522
0,0.033712,0.073668,0.036069,0.026115,0.037753,0.706134,0.058586,0.027963
0,0.009064,0.020933,0.020706,0.012812,0.007356,0.874492,0.047878,0.006759
0,0.327679,0.638166,0.006772,0.004991,0.003473,0.008011,0.007967,0.002941


In [28]:
results 

[[0.3906252384185791,
  0.18486405909061432,
  0.15781289339065552,
  0.15184199810028076,
  0.11485578119754791],
 [0.4453665018081665,
  0.1640147864818573,
  0.15509536862373352,
  0.1480935662984848,
  0.08742983639240265],
 [0.4492569863796234,
  0.15557406842708588,
  0.14863286912441254,
  0.1389198750257492,
  0.10761624574661255],
 [0.9508629441261292,
  0.02037367969751358,
  0.012606591917574406,
  0.008919031359255314,
  0.007237814832478762],
 [0.9447943568229675,
  0.01952427811920643,
  0.014389385469257832,
  0.011278887279331684,
  0.010013055987656116],
 [0.45752954483032227,
  0.17691218852996826,
  0.15424592792987823,
  0.1239752471446991,
  0.08733710646629333],
 [0.4979049563407898,
  0.18130658566951752,
  0.143904909491539,
  0.1061091348528862,
  0.07077440619468689],
 [0.34540992975234985,
  0.2130848467350006,
  0.15827402472496033,
  0.1490149348974228,
  0.13421618938446045],
 [0.25517308712005615,
  0.21822158992290497,
  0.19713623821735382,
  0.18957430

In [13]:
results 

{'sequence': 'final particle',
 'labels': ['language', 'water', 'weather', 'scenery', 'animals', 'human'],
 'scores': [0.6986836791038513,
  0.11770176887512207,
  0.055702559649944305,
  0.04755160212516785,
  0.045752473175525665,
  0.03460792452096939]}